In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns

In [21]:
train = pd.read_csv("I:/train.csv")
test = pd.read_csv("I:/test.csv")
train_notarget = train.drop(["Survived"], axis=1)
complete = train_notarget.append(test, ignore_index=True)

# Veri temizliği

#Yaştaki NaN değerleri veri setinin yaş ortalamasıyla doldurmak ideal gibi

#Kabin kolonunu düşürmek gerek, herhangi bir imputation metodu uygulamak için çok az kullanılabilir veri var

#Embarked kolonundaki iki NaN satırı düşürmek gerek
train.info()

train.head(15)

train.Embarked.value_counts()

train.dropna(subset=["Embarked"], inplace=True)
train.drop("Cabin", axis=1, inplace=True)

#Yaş sütunu hariç her sütunda 889 satır var, Age sütununu da ortalamalarla doldurunca 889 olacak
train.info()

#Yaş sütununu da ortalama yaşı kullanarak doldurdum
mean_age = train.Age.mean()
train.fillna(value=mean_age, inplace=True)
train.info()

#Object veri tiplerini int64'e çevirerek matematiksel işlem yapılabilecek hale getirdim.
train.replace({"male":0, "female":1}, inplace = True)
train.replace({"S":0, "C":1, "Q":2}, inplace = True)

#Yolcu isimleri ve bilet kodları algoritma tarafından kullanılabilecek sayısal girdilere çevrilemez, bu yüzden düşürülmesi gerek.
#Ayrıca PassengerId sütununun da her ne kadar int64 olsa da yorumlanabilir (en azından benim yorumlayabileceğim) bir sayısal değer olmamasından ötürü düşürülmesi şart.
train.drop(["Ticket", "PassengerId"], axis=1, inplace=True)

#Veri temiz ve işlenmeye hazır duruyor.
train.info()
train.columns

# Feature engineering

#Boş bir küme oluşturup train veri setindeki tüm isimleri tarayıp virgülle ayrıldıktan sonra
#ikinci sırada gelen sözcüğü bu boş kümeye eklettim. Yani kişinin unvanını. Ardından unvanın
#sonundaki noktayı kaldırdım.
titles = set()
for name in train["Name"]:
    titles.add(name.split(",")[1].split(".")[0].strip())

#Kümedeki unvanları gemi memurları, asiller, kadın ve erkekler olacak şekilde dictionary içerisinde
#sınıflara ayırdım.
title_dict = {'Capt':"Officer",
 'Col':"Officer",
 'Don':"Royal",
 'Dr':"Officer",
 'Jonkheer':"Royal",
 'Lady':"Royal",
 'Major':"Officer",
 'Master':"Royal",
 'Miss':"Mrs",
 'Mlle':"Mrs",
 'Mme':"Mrs",
 'Mr':"Mr",
 'Mrs':"Mrs",
 'Ms':"Mrs",
 'Rev':"Officer",
 'Sir':"Royal",
 'the Countess':"Royal"}

#Ardından bu sınıfları veri setindeki unvanlara uygulayarak title adında yeni bir feature kolonu oluşturdum.
train["title"] = train["Name"].map(lambda name: name.split(",")[1].split(".")[0].strip())
train["title"] = train["title"].map(title_dict)
train.title.value_counts()
train.replace({"Mr":1, "Mrs":2, "Royal":3, "Officer":4}, inplace=True)

#Kadınlar ve çocuklar daha fazla hayatta kaldığından 
#yaş ortalamasının altındaki kadınları gösteren bir kolon
#belki bir şeyler anlatabilir.
train["young_female"] = (train["Age"] <= train.Age.mean()) & (train["Sex"] == 1)
train["young_female"] = train["young_female"].astype("int64")
train.young_female.isna().value_counts()

#Yukarıdaki hücre ile aynı işlem sayılır, yalnızca "ve" operatörünü "veya" ile
#değiştirdim. Yani gemidekilerin ortalama yaşında ya da ortalamadan daha genç
#kişiler veya kadın olanları barındıran bir kolon ekledim. 
train["young_or_female"] = (train["Age"] <= train.Age.mean()) | (train["Sex"] == 1)
train["young_or_female"] = train["young_or_female"].astype("int64")
train.young_or_female.isna().value_counts()

#Kişilerin gemide kaç akrabası (çocuk ve eş/kardeş) olduğunu gösteren bir kolon oluşturdum 
train["family_count"] = train["SibSp"] + train["Parch"]
train["family_count"].astype("int64")
train.family_count.value_counts()

#Gemide akrabası olanların yaşam oranıyla alakalı bir farklılık olabilir.
train["has_family"] = (train["SibSp"] + train["Parch"]) > 0
train["has_family"].astype("int64")
train.has_family.value_counts()

#Gemide akrabası olmayanların da yaşam oranıyla alakalı bir farklılık olabilir.
train["no_family"] = (train["SibSp"] + train["Parch"]) <= 0
train["no_family"].astype("int64")
train.has_family.value_counts()

#Gemideki ortalama aile bireyi sayısının üzerinde akrabası olanları gösteren bir kolon oluşturdum.
train["crowded_family"] = train["family_count"] > train["family_count"].mean()
train["crowded_family"].astype("int64")
train.crowded_family.value_counts()

#Yolcuların sınıfı ve yaşları arasında negatif bir korelasyon var, yani genç yolcular genelde
#daha alt sınıflardan bilet almışlar. Buna karşın, genç yolcuların gemiden öncelikli biçimde
#tahliye edildiği verisine de sahibiz. Bu çelişen iki girdiyi tek bir kolonda topladım.
train["age_and_class"] = (train["Age"] <= train.Age.mean()) | (train["Pclass"] == 1)
train["age_and_class"] = train["age_and_class"].astype("int64")
train.age_and_class.isna().value_counts()

#Gemiye bindikleri yere göre en çok ölüm Southampton'dan çıkmış. Southampton'dan
#binenleri gösteren bir kolon oluşturdum.
train["embarkloc"] = (train["Embarked"] == 0)
train["embarkloc"] = train["embarkloc"].astype("int64")
train.embarkloc.isna().value_counts()

#Yolcuların sınıfı ve yaşları arasında negatif bir korelasyon var, yani genç yolcular genelde
#daha alt sınıflardan bilet almışlar. Buna karşın, genç yolcuların gemiden öncelikli biçimde
#tahliye edildiği verisine de sahibiz. Bu çelişen iki girdiyi tek bir kolonda topladım.
train["high_fare"] = (train["Fare"] >= train.Fare.mean()) | (train["Pclass"] == 1)
train["high_fare"] = train["high_fare"].astype("int64")
train.high_fare.isna().value_counts()